In [1]:
import header

import numpy as np
import pandas as pd
import spacy

import warnings
import time
import pickle
import os
import nltk

from nltk.tokenize import word_tokenize
from joblib import dump, load
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

In [17]:
fast_coeff = 10

X_train = pd.read_csv(
        "/Users/welto/Library/CloudStorage/OneDrive-CentraleSupelec/2A/CASA/RakutenPjct/data/X_train_update.csv",
        sep=',',
        usecols=lambda column: column not in [
            'Unnamed: 0',
            'imageid',
            'description'
        ]
    )
X_test_challenge = pd.read_csv(
    "/Users/welto/Library/CloudStorage/OneDrive-CentraleSupelec/2A/CASA/RakutenPjct/data/X_test_update.csv",
    sep=',',
    usecols=lambda column: column not in [
        'Unnamed: 0',
        'imageid',
        'description'
    ]
)
Y_train = pd.read_csv(
    "/Users/welto/Library/CloudStorage/OneDrive-CentraleSupelec/2A/CASA/RakutenPjct/data/Y_train_CVw08PX.csv",
    sep=',',
    usecols=lambda column: column != 'Unnamed: 0'
)

X_train = X_train['designation'][:X_train.shape[0]//fast_coeff].tolist()
X_test_challenge = X_test_challenge['designation'][:X_test_challenge.shape[0]//fast_coeff].tolist()

Y_train = Y_train['prdtypecode'][:Y_train.shape[0]//fast_coeff].tolist()

In [25]:
nltk.download("punkt")

X_train_clean = []
X_test_challenge_clean = []

a = len(X_train)
b = len(X_test_challenge)

header.progress_bar(
    0,
    a,
    prefix='Progress:',
    suffix='Complete',
    length=50
)

preprocessing_start_time = time.time()

for k in range(a):
    tokens = word_tokenize(
        header.normalize_accent(X_train[k].lower()),
        language='french'
    )
    X_train_clean.append(tokens)
    header.progress_bar(
        k + 1,
        a,
        prefix='X_train_raw_designation_clean :',
        suffix='Complete',
        length=50
    )

for k in range(b):
    tokens = word_tokenize(
        header.normalize_accent(X_test_challenge[k].lower()),
        language='french'
    )
    X_test_challenge_clean.append(tokens)
    header.progress_bar(
        k + 1,
        b,
        prefix='X_test_raw_designation_clean :',
        suffix='Complete',
        length=50
    )

X_train_strings = [' '.join(tokens) for tokens in X_train_clean]
X_test_strings = [' '.join(tokens) for tokens in X_test_challenge_clean]

preprocessing_end_time = time.time()
preprocessing_time_h, preprocessing_time_min, preprocessing_time_s = header.convert_seconds(
    preprocessing_end_time - preprocessing_start_time)

print(f"Preprocessed in {int(preprocessing_time_h)}h {int(preprocessing_time_min)}min {int(preprocessing_time_s)}s")

X_train_raw_designation_clean : |██████████████████████████████████████████████████| 100.0% Complete
X_test_raw_designation_clean : |██████████████████████████████████████████████████| 100.0% Complete


[nltk_data] Downloading package punkt to /Users/welto/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


AttributeError: 'str' object has no attribute 'is_punct'

In [ ]:
X_train_strings

In [31]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import time

nltk.download("punkt")
nltk.download("stopwords")
stop_words = set(stopwords.words('french'))

X_train_clean = []
X_test_challenge_clean = []

a = 10 #len(X_train)
b = 10 #len(X_test_challenge)

def progress_bar(current, total, prefix='', suffix='', length=50):
    percent = ("{0:.1f}").format(100 * (current / float(total)))
    filled_length = int(length * current // total)
    bar = '█' * filled_length + '-' * (length - filled_length)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end='\r')
    if current == total:
        print()

preprocessing_start_time = time.time()

for k in range(a):
    tokens = word_tokenize(header.normalize_accent(X_train[k].lower()), language='french')
    tokens = [word for word in tokens if word not in string.punctuation and word not in stop_words]
    X_train_clean.append(tokens)
    progress_bar(k + 1, a, prefix='X_train_raw_designation_clean :', suffix='Complete', length=50)

for k in range(b):
    tokens = word_tokenize(header.normalize_accent(X_test_challenge[k].lower()), language='french')
    tokens = [word for word in tokens if word not in string.punctuation and word not in stop_words]
    X_test_challenge_clean.append(tokens)
    progress_bar(k + 1, b, prefix='X_test_raw_designation_clean :', suffix='Complete', length=50)

X_train_strings = [' '.join(tokens) for tokens in X_train_clean]
X_test_strings = [' '.join(tokens) for tokens in X_test_challenge_clean]

preprocessing_end_time = time.time()
preprocessing_time_h, preprocessing_time_min, preprocessing_time_s = header.convert_seconds(
    preprocessing_end_time - preprocessing_start_time)

print(f"Preprocessed in {int(preprocessing_time_h)}h {int(preprocessing_time_min)}min {int(preprocessing_time_s)}s")


X_train_raw_designation_clean : |██████████████████████████████████████████████████| 100.0% Complete
X_test_raw_designation_clean : |██████████████████████████████████████████████████| 100.0% Complete
Preprocessed in 0h 0min 0s


[nltk_data] Downloading package punkt to /Users/welto/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/welto/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [32]:
X_train_strings

['olivia personalisiertes notizbuch 150 seiten punktraster ca din a5 rosen-design',
 "journal arts 133 28/09/2001 l'art marche salon d'art asiatique a paris jacques barrere francois perrier reforme ventes encheres publiques sna fete cent ans",
 'grand stylet ergonomique bleu gamepad nintendo wii u speedlink pilot style',
 'peluche donald europe disneyland 2000 marionnette a doigt',
 'guerre tuques',
 'afrique contemporaine 212 hiver 2004 dossier japon afrique',
 'christof e bildungsprozessen auf der spur',
 'conquerant sept cahier couverture polypro 240 x 320 mm 96 pages 90g seyes incolore',
 'puzzle scooby-doo poster 2x35 pieces',
 'tente pliante v3s5-pro pvc blanc 3 x 4m50 longueur 4m50 largeur 3 blanc h']

In [33]:
np.linspace(5, 10)

array([ 5.        ,  5.10204082,  5.20408163,  5.30612245,  5.40816327,
        5.51020408,  5.6122449 ,  5.71428571,  5.81632653,  5.91836735,
        6.02040816,  6.12244898,  6.2244898 ,  6.32653061,  6.42857143,
        6.53061224,  6.63265306,  6.73469388,  6.83673469,  6.93877551,
        7.04081633,  7.14285714,  7.24489796,  7.34693878,  7.44897959,
        7.55102041,  7.65306122,  7.75510204,  7.85714286,  7.95918367,
        8.06122449,  8.16326531,  8.26530612,  8.36734694,  8.46938776,
        8.57142857,  8.67346939,  8.7755102 ,  8.87755102,  8.97959184,
        9.08163265,  9.18367347,  9.28571429,  9.3877551 ,  9.48979592,
        9.59183673,  9.69387755,  9.79591837,  9.89795918, 10.        ])

[nltk_data] Downloading package punkt to /Users/welto/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/welto/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



KeyboardInterrupt: 

In [51]:
import header

import string
import warnings
import time
import pickle
import os

import numpy as np
import pandas as pd
import nltk

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from joblib import dump, load
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

def download_nltk_data():
    if not nltk.data.find('tokenizers/punkt'):
        nltk.download("punkt")
    if not nltk.data.find('corpora/stopwords'):
        nltk.download("stopwords")

def load_data(fast_coeff : int):
    X_train = pd.read_csv(
        "/Users/welto/Library/CloudStorage/OneDrive-CentraleSupelec/2A/CASA/RakutenPjct/data/X_train_update.csv",
        sep=',',
        usecols=lambda column: column not in [
            'Unnamed: 0',
            'imageid',
            'description'
        ]
    )
    X_test_challenge = pd.read_csv(
        "/Users/welto/Library/CloudStorage/OneDrive-CentraleSupelec/2A/CASA/RakutenPjct/data/X_test_update.csv",
        sep=',',
        usecols=lambda column: column not in [
            'Unnamed: 0',
            'imageid',
            'description'
        ]
    )
    Y_train = pd.read_csv(
        "/Users/welto/Library/CloudStorage/OneDrive-CentraleSupelec/2A/CASA/RakutenPjct/data/Y_train_CVw08PX.csv",
        sep=',',
        usecols=lambda column: column != 'Unnamed: 0'
    )

    X_train = X_train['designation'][:X_train.shape[0] // fast_coeff].tolist()
    X_test_challenge = X_test_challenge['designation'][:X_test_challenge.shape[0] // fast_coeff].tolist()

    Y_train = Y_train['prdtypecode'][:Y_train.shape[0] // fast_coeff].tolist()

    return X_train, X_test_challenge, Y_train

def preprocess_text(text):
    download_nltk_data()

    stop_words = set(stopwords.words('french'))

    tokens = word_tokenize(
        header.normalize_accent(
            text.lower()
        ),
        language='french'
    )
    tokens = [word for word in tokens if word not in string.punctuation and word not in stop_words]

    return ' '.join(tokens)

def train_model(
        X_train,
        Y_train
):
    params = {
        'C': 8.071428571428571,
        'gamma': 0.1,
        'kernel': 'rbf'
    }
    pipeline = Pipeline([
        (
            'preprocessor',
            TfidfVectorizer(
                preprocessor=preprocess_text
            )
        ),
        (
            'classifier',
            SVC(
                C=params['C'],
                gamma=params['gamma'],
                kernel=params['kernel']
            )
        )
    ])

    pipeline.fit(
        X_train,
        Y_train
    )

    return pipeline

def main(fast_coeff : int):
    exec_time_start = time.time()
    warnings.filterwarnings("ignore")

    X_train, X_test_challenge, Y_train = load_data(fast_coeff)

    model = train_model(X_train, Y_train)

    # Access the preprocessed data
    X_train_preprocessed = model.named_steps['preprocessor'].transform(X_train)

    # Print out the preprocessed data for the first few samples
    for i in range(5):  # Print the first 5 samples
        print(f"Original text: {X_train[i]}")
        print(f"Preprocessed text: {X_train_preprocessed[i]}\n")

    Y_pred = model.predict(X_test_challenge)

    header.Save_label_output(Y_pred, len(X_train))

    exec_time_end = time.time()
    exec_time_h, exec_time_min, exec_time_s = header.convert_seconds(exec_time_end - exec_time_start)

    print(
        f"Executed in {int(exec_time_h)}h {int(exec_time_min)}min {int(exec_time_s)}s"
    )

if __name__ == "__main__":
    main(fast_coeff=1000)




Original text: Olivia: Personalisiertes Notizbuch / 150 Seiten / Punktraster / Ca Din A5 / Rosen-Design
Preprocessed text:   (0, 539)	0.3037767415332776
  (0, 523)	0.3037767415332776
  (0, 498)	0.3037767415332776
  (0, 450)	0.3037767415332776
  (0, 423)	0.3037767415332776
  (0, 415)	0.3037767415332776
  (0, 216)	0.3037767415332776
  (0, 211)	0.2778433246170265
  (0, 143)	0.3037767415332776
  (0, 80)	0.3037767415332776
  (0, 17)	0.3037767415332776

Original text: Journal Des Arts (Le) N° 133 Du 28/09/2001 - L'art Et Son Marche Salon D'art Asiatique A Paris - Jacques Barrere - Francois Perrier - La Reforme Des Ventes Aux Encheres Publiques - Le Sna Fete Ses Cent Ans.
Preprocessed text:   (0, 613)	0.20310216736249212
  (0, 554)	0.20310216736249212
  (0, 530)	0.20310216736249212
  (0, 511)	0.20310216736249212
  (0, 495)	0.20310216736249212
  (0, 449)	0.20310216736249212
  (0, 438)	0.16391899568759463
  (0, 372)	0.20310216736249212
  (0, 334)	0.20310216736249212
  (0, 321)	0.203102167362492